In [12]:
from dotenv import load_dotenv
load_dotenv()

from custom_gmail_reader import CustomGmailReader

# Instantiate the CustomGmailReader
loader = CustomGmailReader(
    query="",
    max_results=50,
    results_per_page=10,
    service=None
)

# Load the emails
documents = loader.load_data()

# Print email information
print(f"Number of documents: {len(documents)}")
for i, doc in enumerate(documents[:20]):
    print(f"Document {i+1}:")
    print(f"To: {doc.metadata.get('to', 'N/A')}")
    print(f"From: {doc.metadata.get('from', 'N/A')}")
    print(f"Subject: {doc.metadata.get('subject', 'N/A')}")
    print(f"Date: {doc.metadata.get('date', 'N/A')}")
    print(f"Content snippet: {doc.text[:1000]}...")
    print("=" * 50)

Number of documents: 50
Document 1:
To: Abhinav Tekumalla <abhinav.tekumalla@gmail.com>
From: Gilroy Gardens Tickets <etickets@gilroygardens.org>
Subject: Gilroy Gardens Order #20240921-1902395
Date: Sat, 21 Sep 2024 10:47:40 -0700
Content snippet: ...
Document 2:
To: Abhinav Tekumalla <abhinav.tekumalla@gmail.com>
From: Gilroy Gardens Tickets <etickets@gilroygardens.org>
Subject: Gilroy Gardens Order #20240921-1901920
Date: Sat, 21 Sep 2024 09:17:39 -0700
Content snippet: ...
Document 3:
To: abhinav.tekumalla@gmail.com
From: Morning Brew <crew@morningbrew.com>
Subject: ☕ Nuclear revival
Date: Sat, 21 Sep 2024 05:45:11 -0400 (EDT)
Content snippet: An infamous power plant will serve Microsoft’s energy needs… <a href="http://www.morningbrew.com"><img src="https://link.morningbrew.com/img/5ca7bc60fc942d4513e3128blww56.ixy/a902c185.gif" alt="" border="0" /></a>

September 21, 2024 View Online <https://links.morningbrew.com/c/pMw?mbcid=36806586.24550&mid=9130d4407c0e66c097cfa9d99530abae&mbu

In [10]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler
import os
from dotenv import load_dotenv
 
# Load environment variables
load_dotenv()
 
langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST")
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

# # Create index
index = VectorStoreIndex.from_documents(documents)

# Create retriever
retriever = VectorIndexRetriever(index=index)

# Create query engine
query_engine = RetrieverQueryEngine(retriever=retriever)

# Example query
response = query_engine.query("Summarize The last email from Brandie")
print(response)


Brandie's last email requested feedback on a notice with revisions and sought approval to proceed with sending it out to the membership by the end of the week.


In [11]:
response = query_engine.query("How many emails have I received from Brandie in the last week?")
print(response)

You have received two emails from Brandie in the last week.


In [13]:
response = query_engine.query("When do I have reservations for Gilroy gardens?")
print(response)

You have reservations for Gilroy Gardens on Saturday, September 21, 2024.


In [9]:
response = query_engine.query("Summarize the latest email from Morning Brew")
print(response)

The latest email from Morning Brew discusses various news items including OpenAI's funding round, Disney's response to a hack, failures in the Secret Service leading to an attempted assassination, a sheriff in Kentucky charged with murder, and a political reporter being put on leave. Additionally, the email includes recommendations for watching an anthropologist explain work's impact on society and visualizations of ship movements based on data transmissions.


In [14]:
response = query_engine.query("What are the amazon returns that I need to take care of?")
print(response)

You need to take care of the return of a Kids Tennis Racket for Junior that you requested with Amazon.
